In [1]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
 
from pytrends.request import TrendReq
import matplotlib.pyplot as plt

In [2]:
final_dataframe = pd.read_csv('./data/sells_time_series.csv', index_col=0)
final_dataframe.index = pd.to_datetime(final_dataframe.index)

In [51]:
def return_series(df, name, key_words = None):
    """ 
    :param df: final_dataframe.
    :param name: nazwa wybrenej grupy (klastra) produktów
    :param key_words: key_words to lista słów dla których ma być wyszukany trend. Jeśli nie podacie nic, to domyślnie 
                      do key_words trafi nazwa produktów name
    """
    
    if key_words is None:
        key_words = [name]

    pytrends = TrendReq()
    pytrends.build_payload(kw_list = key_words, timeframe='2018-02-01 2021-02-28', geo='PL')
    trends = pytrends.interest_over_time().resample('MS').sum().drop('isPartial', axis = 1)[name]
    
    sells = df[name]
    return sells, trends

In [52]:
sells, trends = return_series(df = final_dataframe, name ='lampa')

In [48]:
def lag_transform(s, lags_name, lags = 12):
    """
    :param s: pandas.Series
    :param lag: ile kroków do tyłu
    """
    
    new_rows = []
    window = [i for i in range(len(s) - lags + 1)]
    for w in window:
        new_rows.append(s[w : w + 12].values.tolist())
    
    col_names = [lags_name + '_lag' + str(i) for i in range(lags, 0, -1)]
    lagged_data = pd.DataFrame(new_rows, columns = col_names)
    return lagged_data

In [57]:
y = sells[11:].reset_index()
y.columns = ['Data', 'sprzedaż']
frames = [y, lag_transform(sells, 'sprzedaż'), lag_transform(trends, 'trendy')]
df = pd.concat(frames, axis = 1)
df

,Data,sprzedaż,sprzedaż_lag12,sprzedaż_lag11,sprzedaż_lag10,sprzedaż_lag9,sprzedaż_lag8,sprzedaż_lag7,sprzedaż_lag6,sprzedaż_lag5,...,trendy_lag10,trendy_lag9,trendy_lag8,trendy_lag7,trendy_lag6,trendy_lag5,trendy_lag4,trendy_lag3,trendy_lag2,trendy_lag1
0,2019-01-01,24,0,0,0,0,0,0,2,6,...,256,180,195,279,236,284,256,294,358,263
1,2019-02-01,46,0,0,0,0,0,2,6,31,...,180,195,279,236,284,256,294,358,263,237
2,2019-03-01,94,0,0,0,0,2,6,31,50,...,195,279,236,284,256,294,358,263,237,305
3,2019-04-01,124,0,0,0,2,6,31,50,38,...,279,236,284,256,294,358,263,237,305,218
4,2019-05-01,119,0,0,2,6,31,50,38,24,...,236,284,256,294,358,263,237,305,218,213
5,2019-06-01,75,0,2,6,31,50,38,24,46,...,284,256,294,358,263,237,305,218,213,245
6,2019-07-01,64,2,6,31,50,38,24,46,94,...,256,294,358,263,237,305,218,213,245,230
7,2019-08-01,70,6,31,50,38,24,46,94,124,...,294,358,263,237,305,218,213,245,230,248
8,2019-09-01,117,31,50,38,24,46,94,124,119,...,358,263,237,305,218,213,245,230,248,323
9,2019-10-01,129,50,38,24,46,94,124,119,75,...,263,237,305,218,213,245,230,248,323,266
